In [1]:
import discord
import os
import asyncio
from discord.ext import commands
from dotenv import load_dotenv
#importing libraries
import nltk
import sklearn
import numpy as np
from nltk.corpus.reader.tagged import sent_tokenize
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
stopwords= stopwords.words('english')

pool1= ['where is your website located. where is your website. where is your website link']
pool2=['can i join even though i am not in stats/data sciene/cs major. how inclusive is your club']
pool3= ['get to know the officer team. who are the officers and what do they do. who is in the officer team']
pool4=['what has been accomplished so far. greatest accomplishment?']
pool5= ['what do you do in the club? what happens in the club?']
pool6=['club\’s goal? what is the purpose of this club? future goals?']
pool7=['what type of events does your club offer? what events does your club have? any cool events?']
pools=[pool1, pool2, pool3, pool4, pool5, pool6, pool7]

reply= {"Our website can be found here: https://davisdsc.com/": pool1, "We accept all majors.": pool2, "Our current officer team can be found in this part of our website: https://davisdsc.com/about": pool3, "All our accomplishments can be found in our GitHub page: https://github.com/Davis-Data-Science-Club": pool4, "Each quarter we offer workshops about data science topics, talks with industry veterans, project involvement, and much more": pool5,
        "We are on a mission to foster a supportive community centered around developing technical skill sets, career building through industry guest speakers, and enhancing student body engagement." : pool6,
        "Each quarter we offer workshops about data science topics, talks with industry veterans, project involvement, and much more.": pool7}

load_dotenv()

DISCORD_TOKEN = os.getenv("DISCORD_TOKEN")
ROLE_ID = os.getenv("ROLE_ID")

#Specify what type of access the bot will have. In our case, all.
intents = discord.Intents.all()
intents.members = True

#Creates a new discord Bot
bot = commands.Bot(command_prefix="!", intents=intents)

#formatting
def format_string(text):
    text = ''.join([word for word in text if word not in string.punctuation])
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stopwords])
    
    return text


def nltk_answer(ques, pools, reply):
    sToken = nltk.sent_tokenize(ques)

    tv = TfidfVectorizer()
    sim_list = []
    for i in pools:
        #formating user question and the question pools
        format_ques=list(map(format_string, sToken))
        format_pool=list(map(format_string, i))
        
        #vectorizing user question and the question pools
        pool_vectorizer= tv.fit_transform(format_pool)
        pool_vectors= pool_vectorizer.toarray()

        ques_vectorize= tv.transform(format_ques)
        ques_vector= ques_vectorize.toarray()

        #cosine similarity
        similarity= sklearn.metrics.pairwise.cosine_similarity(ques_vector, pool_vectors)
        max_sim= np.ndarray.max(similarity)
        sim_list.append(max_sim)

    #variable for the maximum similarity found
    maxSim= max(sim_list)
    keyl= list(reply.keys())
    vall= list(reply.values())

    #tried something for edge cases and accuracy
    if maxSim <= 0.5: #edge case: when question does not match any pool
        return None
    else: #for question that match pools closely
        question= pools[sim_list.index(max(sim_list))] #matching user question to appropriate pool
        answer= keyl[vall.index(question)]
        return answer


#Event handler that prints "Logged in" when the discord bot is up and running.
@bot.event
async def on_ready():
    print(f'Logged in as {bot.user}')

ratings = {}

#placeholder function for nltk portion
def example_function(input):
    if input == "Freeze!":
        return
    return input

#Bot command that is triggered using !q, will call the function and output a response. 
@bot.command()
async def q(ctx, *, question):
    #response = example_function(question)
    response = nltk_answer(question, pools, reply)
    if response is None:
        await ctx.send("Sorry, I couldn't find an answer to that question.")
        role = ctx.guild.get_role(ROLE_ID)
        await ctx.send("Maybe an " + role.mention + " can help you out.")
    else:
        await ctx.send(response)
        # Ask the user to rate the response
        await ctx.send("Please rate my response from 1 to 10.")

        def check(message):
            return message.author == ctx.author and message.channel == ctx.channel and message.content.isdigit()

        # Wait for the user's rating
        try:
            rating = await bot.wait_for("message", check=check, timeout=10.0)
            rating_value = int(rating.content)
            if rating_value < 1 or rating_value > 10:
                await ctx.send("Invalid rating. Please enter a number between 1 and 10.")
            else:
                # Store the question, response, and rating
                ratings[(ctx.author.name, question, response)] = rating_value
                await ctx.send("Thank you for your rating.")
        except asyncio.TimeoutError:
            await ctx.send("You did not enter a rating. Rating will not be recorded.")

        # Write the ratings to a file
        with open("ratings.txt", "a") as file:
            file.write(f"{ctx.author.name}, {question}, {response}, {rating_value}\n")

# Start the event loop and keep it running
loop = asyncio.get_event_loop()
loop.create_task(bot.start(DISCORD_TOKEN))
loop.run_forever()


RuntimeError: This event loop is already running

Logged in as HelloBot#5519
